# Hierarchical Clustering Analysis with Max Linkage
As described in this paper: http://www.econ.upf.edu/~michael/stanford/maeb7.pdf

### Basic Description

Hierarchical cluster analysis is an unsupervised clustering algorithm that tries to build a hierarchy of clusters. The datasets I will be working with all have a diagonal of zeroes. The A sample is represented by the diagonal. The values are also reflected across the diagonal. Each value is the dissimilarity value of two samples, A and the sample in question. For example, the first column and first row sample would be the dissimilarity value between A and A (0). The first column and second row sample would be the dissimilarity value between A and B (0.5). The grid including A, B, C, etc. is shown in the paper.

The algorithm starts by finding the minimum value in the dataset. There should be at least 2 since the data is reflected across the diagonal. Then the value on the left side of the diagonal searches for the largest value in its row. The value on the left side of the diagonal searches for the largest value in its column. If the dataset is made correctly and the algorithm worked correctly, these two largest values should be the same. The minumum value is replaced with the found largest value. This largest value now represents the dissimilarity value between A and the row and column that were searched through. Then the row and column that were searched through are deleted, and the process begins again. This process is repeated until there is only a two by two square left which houses the maximum dissimilarity value. 

The clustering portion of this algorithm becomes more clear when the deleted rows and columns are summarized in a dendrogram. For example, after the first iteration of this algorithm, the dendrogram would look like exhibit 7.3, page 3 of the given paper. Exhibit 7.6 page 4 shows when a cluster joins another via a node. Exhibit 7.8 page 5 shows the complete dendrogram for the dataset. 

In [1]:
import numpy as np

def maxLinkage(data):
    data=np.array(data)
    iter = 0
    
    #Continue until there are only two columns
    while(len(data[0])>2):
        
        #Remove zeroes from list to find min
        listNoZeroes = np.ma.masked_equal(data, 0.0, copy=False)
        
        #Find minimum value
        least = listNoZeroes.min()
        index = np.where(data==least)
        
        #Change rows to max value between the two rows
        #Notice that the data will always be a square
        for z in range(len(data[0])):
            if(data[index[0][0]][z]!=0):
                data[index[0][0]][z] = max(data[index[0][0]][z], data[index[1][0]][z])
                
        #Change columns to max value between two columns
        for z in range(len(data[0])):
            if(data[z][index[0][0]]!=0):
                data[z][index[0][0]] = max(data[z][index[0][0]], data[z][index[1][0]])
                
        #Delete row and column where min value was
        data = np.delete(data,index[1][0], 0)
        data = np.delete(data,index[1][0], 1)
        
        print('Iteration: ', iter)
        print(data)
        print()
        iter +=1
        
    return data

In [2]:
#Same dataset as in given paper
#Notice that the diagonals are zeroes and the values are reflected across diagonal.

data1 = [
[0, 0.5000, 0.4286, 1.0000, 0.2500, 0.6250, 0.3750],
[0.5000, 0, 0.7143, 0.8333, 0.6667, 0.2000, 0.7778],
[0.4286, 0.7143, 0, 1.0000, 0.4286, 0.6667, 0.3333],
[1.0000, 0.8333, 1.0000, 0, 1.0000, 0.8000, 0.8571],
[0.2500, 0.6667, 0.4286, 1.0000, 0, 0.7778, 0.3750],
[0.6250, 0.2000, 0.6667, 0.8000, 0.7778, 0, 0.7500],
[0.3750, 0.7778, 0.3333, 0.8571, 0.3750, 0.7500, 0]]

In [3]:
print('Original')
print(np.array(data1))
print()
maxLinkage(data1)

Original
[[0.     0.5    0.4286 1.     0.25   0.625  0.375 ]
 [0.5    0.     0.7143 0.8333 0.6667 0.2    0.7778]
 [0.4286 0.7143 0.     1.     0.4286 0.6667 0.3333]
 [1.     0.8333 1.     0.     1.     0.8    0.8571]
 [0.25   0.6667 0.4286 1.     0.     0.7778 0.375 ]
 [0.625  0.2    0.6667 0.8    0.7778 0.     0.75  ]
 [0.375  0.7778 0.3333 0.8571 0.375  0.75   0.    ]]

Iteration:  0
[[0.     0.625  0.4286 1.     0.25   0.375 ]
 [0.625  0.     0.7143 0.8333 0.7778 0.7778]
 [0.4286 0.7143 0.     1.     0.4286 0.3333]
 [1.     0.8333 1.     0.     1.     0.8571]
 [0.25   0.7778 0.4286 1.     0.     0.375 ]
 [0.375  0.7778 0.3333 0.8571 0.375  0.    ]]

Iteration:  1
[[0.     0.7778 0.4286 1.     0.375 ]
 [0.7778 0.     0.7143 0.8333 0.7778]
 [0.4286 0.7143 0.     1.     0.3333]
 [1.     0.8333 1.     0.     0.8571]
 [0.375  0.7778 0.3333 0.8571 0.    ]]

Iteration:  2
[[0.     0.7778 0.4286 1.    ]
 [0.7778 0.     0.7778 0.8333]
 [0.4286 0.7778 0.     1.    ]
 [1.     0.8333 1.     0. 

array([[0., 1.],
       [1., 0.]])

In [4]:
#Matrix with all zeroes in the diagonal
data = [[0, 1, 1, 2, 2, 1],
       [1, 0, 1, 1, 4, 1],
       [1, 1, 0, 2, 2, 0],
       [2, 1, 2, 0, 1, 2],
       [2, 4, 2, 1, 0, 2],
       [1, 1, 0, 2, 2, 0]]

In [5]:
print('Original')
print(np.array(data))
print()
maxLinkage(data)

Original
[[0 1 1 2 2 1]
 [1 0 1 1 4 1]
 [1 1 0 2 2 0]
 [2 1 2 0 1 2]
 [2 4 2 1 0 2]
 [1 1 0 2 2 0]]

Iteration:  0
[[0 1 2 4 1]
 [1 0 2 2 0]
 [2 2 0 1 2]
 [4 2 1 0 2]
 [1 0 2 2 0]]

Iteration:  1
[[0 2 4 1]
 [2 0 1 2]
 [4 1 0 2]
 [1 2 2 0]]

Iteration:  2
[[0 2 4]
 [2 0 1]
 [4 1 0]]

Iteration:  3
[[0 4]
 [4 0]]



array([[0, 4],
       [4, 0]])